In [ ]:
def rf_param_function(trial):
    params = {
        "n_estimators":trial.suggest_int("n_estiamtors",100,500,step=100)
    }
    return params

def outer_objective(clf,param_function):
    def inner_objective(trial):
        params = param_function(trial)
        model = clf.set_params(**params).fit(x_train,y_train)

        model.predict(x_val)

        f1 = clf.predict(x_val)

        return f1
    return inner_objective

study = optuna.create_study(direction="maximize")
study.optimize(outer_objective(RandomForestClassifier(),rf_param_function))